<a href="https://colab.research.google.com/github/etgins/Mice_ASD_Detection/blob/main/audio_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------------------------
Written by Itamar Ginsberg & Alon Schreuer, October 2021



# **1. Import data and split labels**

---

In [53]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## COLAB WORK
## ITAMAR:
dataset = pd.read_excel('/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.xlsx')
## ALON:
# dataset = pd.read_csv("/content/drive/MyDrive/total_data.csv")
## ITAMAR:
#dataset.to_csv (r'/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.csv', index = None, header=True)
# dataset.dropna(axis = 0, how='any')

## ALON:
#dataset.to_csv("/content/drive/MyDrive/total_data.csv", index = None, header=True)
#dataset = pd.read_csv("/content/drive/MyDrive/total_data.csv", error_bad_lines=False)

# extract only the relevant columns / features
X = dataset[["Name", "Start Point (Hz)", "End Point (Hz)", "Duration (s)", "Syllable number", "Offspring Genotype"]]
# clean NaN values from dataset or X
X = X.dropna(axis = 0, how='any')
# print(X)

# create labels
y = X[["Offspring Genotype"]]
print(y)
# encode labels
l1 = LabelEncoder()
l1.fit(y)
y = l1.transform(y)
X[["Offspring Genotype"]] = y
print(X)
# OR use this:
      # convert the target column to categorical
      # col = pd.Categorical(y)
      # y = pd.Series(col.codes)



# convert duration to micro-seconds from 'datetime.time' type to 'timedelta' object
from datetime import datetime
duration_vec = X[["Duration (s)"]]
# print(duration_vec)
duration_vec2 = np.array(duration_vec)
# print(duration_vec2)
duration_vec3 = np.zeros([1, np.size(duration_vec2)])
# print(duration_vec3)
for i in range (len(duration_vec)):
  duration_single = duration_vec2[i,0]
  # print(duration_single)
  duration_vec3[0,i] = int(duration_single.strftime(format = '%f'))
  # print(duration_vec3[0,i])
X[["Duration (s)"]] = np.transpose(duration_vec3)
# print(X)




# X.describe()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     Offspring Genotype
1                    WT
2                    WT
3                    WT
4                    WT
5                    WT
...                 ...
3167                 WT
3170                 HT
3171                 HT
3172                 HT
3173                 HT

[3035 rows x 1 columns]
        Name  Start Point (Hz)  ...  Syllable number Offspring Genotype
1     17470O        57648.8278  ...              3.0                  1
2     17470O        55558.5268  ...              2.0                  1
3     17470O        54513.3763  ...              2.0                  1
4     17470O        55976.5870  ...              2.0                  1
5     17470O        59739.1288  ...              4.0                  1
...      ...               ...  ...              ...                ...
3167  08121P        73793.2416  ...              8.0  

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# **2. Split data for different mice by name - unfinished**

---

In [55]:
# split dataset for each mouse

def take_data_by_name(name_searched, Data, name_list):
  # print(name_list)
  ind = name_list == name_searched
  # print(ind)
  matching_data = Data[ind].reset_index()
  # TODO - drop index column
  
  # print(matching_data)
  return matching_data


# find set of mouse names
name_column = X["Name"]
unique_name_set = set(name_column)
unique_name_list = list(unique_name_set)
# unique_name_list = unique_name_list[0:-1]   # first entry is column name

number_of_mice = len(unique_name_list)
# print(unique_name_list)
# print(number_of_mice, 'mice in data')

# remove nan from list of names
unique_name_list = [x for x in unique_name_list if str(x) != 'nan']
number_of_mice = len(unique_name_list)

# print(unique_name_list)
# print(number_of_mice, 'mice in data')

# **3. Extract features**

---

In [82]:
  # final data is made up of: 
    # avg. start freq. per syllable
    # avg. end freq. per syllable
    # avg. syllable duration per syllable
    # syllable distribution - percentage of each syllable
    # Bandwidth - TBD

num_of_syllables = 10
final_data_size = 4*num_of_syllables
# initialize finalized data variable: mouse_final_data
mouse_final_data = np.zeros([number_of_mice,final_data_size+1]) # the extra column will hold the genotype label

""" 
Work on each mouse separately and calculate all features.
Place the results in mouse_final_data[mouse_idx,:]
"""

for idx in range (0,number_of_mice): 
    mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
    # print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
    # print('matching data:', '\n',mouse_split_data)



    # calculate feature 1 - average start freq for each syllable (Alon)
    ## -------------------------------------------
    # print("\n", "--- CALCULATING FEATURE 1: average start freq for each syllable ---", "\n")
    """
    - take start frequencies grouped by syllable number, and calculate the average
    - fill a 10-long vector with the final result ( = add zeros to non-existing syllables)
    - place in mouse_final_data[0:10]
    """

    start_frequencies = np.array(mouse_split_data.groupby(by = 'Syllable number', as_index=False)["Start Point (Hz)"].mean())
    # print('start_frequencies :','\n',start_frequencies,'\n')
    
    feature1_vec = np.zeros([1,10])
    for i in range(len(start_frequencies)):
      feature1_vec[0,int(start_frequencies[i,0])-1] = start_frequencies[i,1]
    
    # PRINT RESULT:
    # print('each syllable mean start freq:','\n', feature1_vec, '\n')


    mouse_final_data[idx,0:10] = feature1_vec


    # calculate feature 2 - average end freq for each syllable (Alon)
    ## -------------------------------------------
    # print("\n", "--- CALCULATING FEATURE 2: average end freq for each syllable ---", "\n")
    

    """
    ITAMAR:
    - take end frequencies grouped by syllable number, and calculate the average
    - fill a 10-long vector with the final result ( = add zeros to non-existing syllables)
    - place in mouse_final_data[0:10]
    """

    end_frequencies = np.array(mouse_split_data.groupby(by = 'Syllable number', as_index=False)["End Point (Hz)"].mean())
    # print('end_frequencies :','\n',end_frequencies, '\n')
    
    feature2_vec = np.zeros([1,10])
    for i in range(len(end_frequencies)):
      feature2_vec[0,int(end_frequencies[i,0])-1] = end_frequencies[i,1]
    
    # PRINT RESULT:
    # print('each syllable mean end freq:','\n', feature2_vec, '\n')

    mouse_final_data[idx,10:20] = feature2_vec






    ## -------------------------------------------
    # calculate feature 3 - syllable distribution (Itamar):
    ## -------------------------------------------
    # print("\n", "--- CALCULATING FEATURE 3: mouse's syllable distribution ---", "\n")

  # take syll numbers
    syllable_num_row = np.transpose(np.array(mouse_split_data["Syllable number"]))
    # print(syllable_num_row)

  # initialize distribution vector
    distribution = np.zeros([1,10])
    # print(distribution)

  # increment syllable count
    for i in range(len(syllable_num_row)):
      # print("iteration", i)
      syll = int(syllable_num_row[i])
      # print("found syllable", syll)
      # print("old distribution:", distribution)
      distribution[0,syll-1] += 1 
      # print("new distribution:", distribution)

  # normalize histogram to distribution
    distribution = distribution / distribution.sum()
    # if all are zeros, dividing by zero will create nan's - make them zero
    distribution[np.isnan(distribution)] = 0  

    # PRINT RESULT:
    # print("final distribution:", "\n", distribution)

  # transfer results to mouse_final_data
    mouse_final_data[idx,20:30] = distribution




    ## -------------------------------------------
    # calculate feature 4 - average syllable duration for each syllable (Itamar)
    ## -------------------------------------------
    # print("\n", "--- CALCULATING FEATURE 4: average syllable duration for each syllable ---", "\n")
    
    # take the mean of the duration column for each syllable
    mouse_duration = mouse_split_data[["Name", "Syllable number","Duration (s)"]]
    # print(mouse_duration)
    a = mouse_duration.groupby('Syllable number', as_index=False)["Duration (s)"].mean()
    a = np.array(a)
    
    # PRINT RESULT:
    # print("for this mouse, each syllable's mean duration is:","\n", a)

    feature4_vec = np.zeros([1,10])
    # print(np.shape(a))
    for i in range (len(a)):
      # print('iteration ', i)
      # print(int(a[i,0]))
      syllable = int(a[i,0])
      feature4_vec[0,syllable-1] = int(a[i,1])
      # print(" means_vec:", means_vec)
    
    mouse_final_data[idx,30:40] = feature4_vec



    ## -------------------------------------------
    # calculate feature 5 - Bandwidth (???) - TBD
    ## -------------------------------------------
    # print("\n", "--- CALCULATING FEATURE 5: Bandwidth ---", "\n")



    ## -------------------------------------------
    # find mouse's label
    ## -------------------------------------------
    mouse_final_data[idx,-1] = mouse_split_data["Offspring Genotype"].iloc[0]


    # print("\n", "--- FINISHED features, show final data: --- (mouse ", idx, ")" "\n")
    # print(mouse_final_data[idx,:], "\n")


print("\n", "--- FINAL DATA FOR ALL MICE --- ", "\n")
print(mouse_final_data[:,:], "\n")


np.savetxt("processed_data_for_final_classification.csv", X=mouse_final_data, delimiter=",")
# np.save("trial2.csv",mouse_final_data)
!cp "processed_data_for_final_classification.csv" /content/drive/MyDrive/Project_A/Project_A_files



 --- FINAL DATA FOR ALL MICE ---  

[[6.94129079e+04 5.80254787e+04 6.34544756e+04 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [8.54836700e+04 0.00000000e+00 8.48825299e+04 ... 6.87280000e+04
  0.00000000e+00 1.00000000e+00]
 [6.83624883e+04 6.08928857e+04 6.14381733e+04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.95941393e+04 7.10685975e+04 6.22200727e+04 ... 7.57500000e+04
  0.00000000e+00 1.00000000e+00]
 [7.75763640e+04 7.65755532e+04 7.68198741e+04 ... 6.67500000e+04
  0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 4.99147141e+04 7.30473785e+04 ... 6.28330000e+04
  7.80000000e+04 1.00000000e+00]] 



Archive
---

In [ ]:
  """ ALON OLD CODE
  # TODO 
  # extract all syllables per mouse - done
  # create a dictionay with num of syllable as key and all it's start freqs arrange in list as a value
  # for every value in the dictionary compute the average
  # to it again in a loop for every mouse such that in the end I'll get an array with num of mouse as a key and the value is the dictionay with the average of start freqs per syllable (*think about better way to arrange the data)
  

  # create a dictionay with num of syllable as key and the sum of it's start freqs 
  # dict = {}
  # start_freq = np.array(mouse_split_data["Start Point (Hz)"]) # array of start_freqs
  # syllable_num = np.array(mouse_split_data["Syllable number"]) # array of sillable num
  # start_freq_check_nan = [~np.isnan(start_freq)] # start_freqs array convert to in_nan (true/false)
  # count = 0
  # for i in syllable_num:
  #   j = [~np.isnan(i)]
  #   if not j:
  #     print("alon")
  #     continue
  #   else:
  #     dict[i] = dict[i] + start_freq[i]
  # print(dict)
  
# print(start_freq)
# print(len(start_freq))
    # get the num of each syllable 

  """